In [82]:
import numpy as np
import pandas as pd

In [83]:
def id(number):
    if len(str(number))==1:
        return '00'+str(number)
    else:
        return '0'+str(number)

In [84]:
foods = []
dexcoms = []
to_return = {}
for i in range(1, 17):
    current_id = id(i)
    foods.append(pd.read_csv(f'data/{current_id}/Food_Log_{current_id}.csv'))
    dexcoms.append(pd.read_csv(f'data/{current_id}/Dexcom_{current_id}.csv'))

In [85]:
foods[12]

,date,time_of_day,time_begin,time_end,logged_food,amount,unit,searched_food,calorie,total_carb,dietary_fiber,sugar,protein,total_fat
0,12/30/2019,16:30,12/30/2019 16:30,16:32,Ice Cream Sandwich,4.23,fluid ounce,Ice Cream Sandwich,284.0,45.0,0.0,22.0,5.1,10.0
1,12/30/2019,18:57,12/30/2019 18:57,19:10,Lasagna (mostly vegetables),335.00,gram,Lasagna,524.0,30.0,3.4,9.8,38.0,28.0
2,12/30/2019,18:57,12/30/2019 18:57,19:08,Red Wine,0.25,cup,Red Wine,50.0,1.5,0.0,0.4,0.0,0.0
3,12/30/2019,19:02,12/30/2019 19:02,19:02,Cashew Nut,6.00,NaN,Cashew Nut,54.0,3.1,0.3,0.5,1.4,4.4
4,12/30/2019,19:27,12/30/2019 19:27,19:28,Chocolate,10.00,gram,Chocolate,54.0,5.9,0.3,5.2,0.8,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1/5/2020,19:45,1/5/2020 19:45,20:05,beet,1.00,NaN,Beet,22.0,5.0,1.0,4.0,0.8,0.1
86,1/5/2020,20:20,1/5/2020 20:20,20:37,tea,NaN,NaN,Tea,2.4,0.7,0.0,0.0,0.0,0.0
87,1/5/2020,20:20,1/5/2020 20:20,20:37,Honey,1.00,teaspoon,Honey,21.0,5.8,0.0,5.8,0.0,0.0
88,1/5/2020,20:40,1/5/2020 20:40,20:45,cookie,8.00,NaN,Cookie,1181.3,157.3,4.8,78.7,12.3,58.7


In [86]:
dexcoms[12]

,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer)
0,1,NaN,FirstName,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,LastName,NaN,013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,DateOfBirth,NaN,1970-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,Device,NaN,NaN,Dexcom G6 Mobile App,iPhone G6,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,Alert,Fall,NaN,NaN,iPhone G6,NaN,NaN,NaN,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1985,1986,2020-06-04 11:51:40,EGV,NaN,NaN,NaN,iPhone G6,127.0,NaN,NaN,NaN,NaN,599985.0
1986,1987,2020-06-04 11:56:40,EGV,NaN,NaN,NaN,iPhone G6,125.0,NaN,NaN,NaN,NaN,600285.0
1987,1988,2020-06-04 12:01:39,EGV,NaN,NaN,NaN,iPhone G6,126.0,NaN,NaN,NaN,NaN,600585.0
1988,1989,2020-06-04 12:06:39,EGV,NaN,NaN,NaN,iPhone G6,118.0,NaN,NaN,NaN,NaN,600885.0


In [87]:
for i in range(16):
    dexcom = dexcoms[i]
    food = foods[i]
    dexcom['time'] = pd.to_datetime(dexcom['Timestamp (YYYY-MM-DDThh:mm:ss)'])
    dexcom = dexcom[dexcom['time'].notna()]
    food['time'] = pd.to_datetime(food['time_begin'])
    food = food[food['time'].notna()]

    #food = food.groupby('time', as_index=False)[['calorie', 'total_carb', 'dietary_fiber', 'sugar', 'protein', 'total_fat']].sum()
    food = food.groupby('time', as_index=False).agg({
        'calorie': 'sum',
        'total_carb': 'sum',
        'dietary_fiber': 'sum',
        'sugar': 'sum',
        'protein': 'sum',
        'total_fat': 'sum',
        'logged_food': list,     # example column to aggregate as list
    })
    dexcom = dexcom.sort_values('time')
    food = food.sort_values('time')

    # Create a results dataframe to store our findings
    results = []
    for _, food_elem in food.iterrows():
        # Find the closest timestamp in df that comes before food_time
        food_time = food_elem['time']
        mask = dexcom['time'] <= food_time
        if not mask.any():
            continue  # Skip if no earlier timestamp exists
        closest_time = dexcom.loc[mask, 'time'].max()
        basis_glucose = dexcom.loc[dexcom['time'] == closest_time, 'Glucose Value (mg/dL)'].iloc[0]
    
        # Find all records within one hour after the closest_time
        one_hour_later = closest_time + pd.Timedelta(hours=1)
        time_window = dexcom[(dexcom['time'] >= closest_time) & (dexcom['time'] <= one_hour_later)]
    
        # Calculate sugar changes relative to the basis
        for _, row in time_window.iterrows():
            results.append({
                'participant_id': id(i+1),
                'food_time': food_time,
                'basis_time': closest_time,
                'current_time': row['time'],
                'basis_glucose': basis_glucose,
                'current_glucose': row['Glucose Value (mg/dL)'],
                'sugar_change': row['Glucose Value (mg/dL)'] - basis_glucose,
                'minutes_from_basis': (row['time'] - closest_time).total_seconds() / 60,
                'calorie': food_elem['calorie'],
                'total_carb': food_elem['total_carb'],
                #'dietary_fiber': food_elem['dietary_fiber'],
                'sugar': food_elem['sugar'],
                'protein': food_elem['protein'],
                #'total_fat': food_elem['total_fat'],
                'logged_food': food_elem['logged_food']
            })
    results_df = pd.DataFrame(results)
    to_return[id(i+1)] = results_df

In [88]:
for i in range(1, 17):
    if i==3:
        continue
    to_return[id(i)].to_csv(f'data/{id(i)}/summary{id(i)}.csv')

In [89]:
male_high = []
for i in [9, 11]:
    male_high.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
male_high_df = pd.concat(male_high).drop('Unnamed: 0', axis=1)
male_high_df.to_csv(f'data/male_high.csv')

In [96]:
male_low = []
for i in [2, 12, 14]:
    male_low.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
male_low_df = pd.concat(male_low).drop('Unnamed: 0', axis=1)
male_low_df.to_csv(f'data/male_low.csv')

In [99]:
female_high = []
for i in [4, 5, 6, 10]:
    female_high.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
female_high_df = pd.concat(female_high).drop('Unnamed: 0', axis=1)
female_high_df.to_csv(f'data/female_high.csv')

In [106]:
female_low = []
for i in [1, 8]:
    female_low.append(pd.read_csv(f'data/{id(i)}/summary{id(i)}.csv'))
female_low_df = pd.concat(female_low).drop('Unnamed: 0', axis=1)
female_low_df.to_csv(f'data/female_low.csv')